In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix
import pickle
from scipy.stats import uniform
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


### evaluation metric = f1 score

#### tfidf via custom functionBySven

In [2]:
!pip install gensim

%run ../functions/vectorize_functions.py

In [6]:
# Load training data
df_train = pd.read_csv("../data/mixed_dataset/train_cleaned.csv")
df_train = df_train.dropna(subset=['tweet_cleaned'])

# TF-IDF Vectorization on training data + saving it 
X_train, X_test, y_train, y_test, tfidf_vectorizer = vectorize_tfidf(df=df_train, text_column="tweet_cleaned", label_column="label")

with open("svm_tfidf_vectorizer_opt.pkl", "wb") as file:
    pickle.dump(tfidf_vectorizer, file)

# Hyperparameter search for SVC-model
param_dist = {
    'kernel': ['sigmoid'],
    'C': uniform(0.01, 1),
    'gamma': uniform(0.01, 0.1),
    'class_weight': [{0: 1, 1: 1}, {0: 1, 1: 3}, {0: 1, 1: 5}]
}

random_search = RandomizedSearchCV(
    SVC(),
    param_distributions=param_dist,
    n_iter=10, 
    scoring='f1',
    cv=2, 
    verbose=1,  
    n_jobs=-1,  
    random_state=42,
    refit=True 
)

random_search.fit(X_train, y_train)

# Retrieing best model found
best_model = random_search.best_estimator_

# Load test data
df_test = pd.read_csv("../data/mixed_dataset/test_cleaned.csv")
df_test = df_test.dropna(subset=['tweet_cleaned'])

# Use vectorizer to transform test data
X_test = tfidf_vectorizer.transform(df_test['tweet_cleaned'])
y_test = df_test["label"]

# Evaluate model on test set
y_test_pred = best_model.predict(X_test)
test_report = classification_report(y_test, y_test_pred, output_dict=True)
cm = confusion_matrix(y_test, y_test_pred)

#--------------------------------------
# Calculate additional metrics and add them to classification report df
accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred, average='weighted')
recall = recall_score(y_test, y_test_pred, average='weighted')
f1 = f1_score(y_test, y_test_pred, average='weighted')

classification_report_df = pd.DataFrame(test_report).transpose()

classification_report_df['accuracy'] = accuracy
classification_report_df['precision'] = precision
classification_report_df['recall'] = recall
classification_report_df['f1_score'] = f1
#----------------------

print("Test Classification Report:")
print(test_report)

# Save best model and hyperparameter search results
with open("svm_tfidf_sigmoid_best_model_opt.pkl", "wb") as file:
    pickle.dump(best_model, file)

results_df = pd.DataFrame(random_search.cv_results_)
results_df.to_csv("svm_tfidf_sigmoid_results_opt.csv", index=False)

# Save configuration of best model
best_model_config = best_model.get_params()
best_model_config_df = pd.DataFrame([best_model_config])
best_model_config_df.to_csv("svm_tfidf_sigmoid_best_model_config_opt.csv", index=False)

# Save confusion matrix
unique_labels = sorted(y_test.unique())
cm_df = pd.DataFrame(cm, 
                     index=[f"Actual_{label}" for label in unique_labels],
                     columns=[f"Predicted_{label}" for label in unique_labels])
cm_df.to_csv("svm_tfidf_sigmoid_confusion_matrix_opt.csv", index=True)

# Save classification report
classification_report_df = pd.DataFrame(test_report).transpose()
classification_report_df.to_csv("svm_tfidf_sigmoid_classification_report_opt.csv", index=True)


In [4]:
# # Load training data
# df_train = pd.read_csv("../data/mixed_dataset/train_cleaned.csv")
# df_train = df_train.dropna(subset=['tweet_cleaned'])

# # TF-IDF Vectorization on training data + saving it 
# X_train, X_test, y_train, y_test, tfidf_vectorizer = vectorize_tfidf(df=df_train, text_column="tweet_cleaned", label_column="label")

# with open("svm_tfidf_linear_vectorizer_opt.pkl", "wb") as file:
#     pickle.dump(tfidf_vectorizer, file)

# Hyperparameter search for SVC-model
param_dist = {
    'kernel': ['linear'],
    'C': uniform(0.01, 1),
    'gamma': uniform(0.01, 0.1),
    'class_weight': [{0: 1, 1: 1}, {0: 1, 1: 3}, {0: 1, 1: 5}]
}

random_search = RandomizedSearchCV(
    SVC(),
    param_distributions=param_dist,
    n_iter=10, 
    scoring='f1',
    cv=2, 
    verbose=1,  
    n_jobs=-1,  
    random_state=42,
    refit=True 
)

random_search.fit(X_train, y_train)

# Retrieing best model found
best_model = random_search.best_estimator_

# Load test data
df_test = pd.read_csv("../data/mixed_dataset/test_cleaned.csv")
df_test = df_test.dropna(subset=['tweet_cleaned'])

# Use vectorizer to transform test data
X_test = tfidf_vectorizer.transform(df_test['tweet_cleaned'])
y_test = df_test["label"]

# Evaluate model on test set
y_test_pred = best_model.predict(X_test)
test_report = classification_report(y_test, y_test_pred, output_dict=True)
cm = confusion_matrix(y_test, y_test_pred)

#--------------------------------------
# Calculate additional metrics and add them to classification report df
accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred, average='weighted')
recall = recall_score(y_test, y_test_pred, average='weighted')
f1 = f1_score(y_test, y_test_pred, average='weighted')

classification_report_df = pd.DataFrame(test_report).transpose()

classification_report_df['accuracy'] = accuracy
classification_report_df['precision'] = precision
classification_report_df['recall'] = recall
classification_report_df['f1_score'] = f1
#----------------------

print("Test Classification Report:")
print(test_report)

# Save best model and hyperparameter search results
with open("svm_tfidf_linear_best_model_opt.pkl", "wb") as file:
    pickle.dump(best_model, file)

results_df = pd.DataFrame(random_search.cv_results_)
results_df.to_csv("svm_tfidf_linear_results_opt.csv", index=False)

# Save configuration of best model
best_model_config = best_model.get_params()
best_model_config_df = pd.DataFrame([best_model_config])
best_model_config_df.to_csv("svm_tfidf_linear_best_model_config_opt.csv", index=False)

# Save confusion matrix
unique_labels = sorted(y_test.unique())
cm_df = pd.DataFrame(cm, 
                     index=[f"Actual_{label}" for label in unique_labels],
                     columns=[f"Predicted_{label}" for label in unique_labels])
cm_df.to_csv("svm_tfidf_linear_confusion_matrix_opt.csv", index=True)

# Save classification report
classification_report_df = pd.DataFrame(test_report).transpose()
classification_report_df.to_csv("svm_tfidf_linear_classification_report_opt.csv", index=True)


Fitting 2 folds for each of 10 candidates, totalling 20 fits
Test Classification Report:
{'0': {'precision': 0.9072230538922156, 'recall': 0.6933527830215662, 'f1-score': 0.7859991569663759, 'support': 34962.0}, '1': {'precision': 0.5159819413092551, 'recall': 0.8217572620074777, 'f1-score': 0.6339231238560098, 'support': 13908.0}, 'accuracy': 0.7298956414978515, 'macro avg': {'precision': 0.7116024976007354, 'recall': 0.757555022514522, 'f1-score': 0.7099611404111928, 'support': 48870.0}, 'weighted avg': {'precision': 0.7958790515635106, 'recall': 0.7298956414978515, 'f1-score': 0.7427195689062374, 'support': 48870.0}}


In [ ]:
# # Load training data
# df_train = pd.read_csv("../data/mixed_dataset/train_cleaned.csv")
# df_train = df_train.dropna(subset=['tweet_cleaned'])

# # TF-IDF Vectorization on training data + saving it 
# X_train, X_test, y_train, y_test, tfidf_vectorizer = vectorize_tfidf(df=df_train, text_column="tweet_cleaned", label_column="label")

# with open("svm_tfidf_poly_vectorizer_opt.pkl", "wb") as file:
#     pickle.dump(tfidf_vectorizer, file)

# Hyperparameter search for SVC-model
param_dist = {
    'kernel': ['poly'],
    'degree': [3, 4, 5],
    'C': uniform(0.01, 1),
    'gamma': uniform(0.01, 0.1),
    'class_weight': [{0: 1, 1: 1}, {0: 1, 1: 3}, {0: 1, 1: 5}]
}

random_search = RandomizedSearchCV(
    SVC(),
    param_distributions=param_dist,
    n_iter=10, 
    scoring='f1',
    cv=2, 
    verbose=1,  
    n_jobs=-1,  
    random_state=42,
    refit=True 
)

random_search.fit(X_train, y_train)

# Retrieing best model found
best_model = random_search.best_estimator_

# Load test data
df_test = pd.read_csv("../data/mixed_dataset/test_cleaned.csv")
df_test = df_test.dropna(subset=['tweet_cleaned'])

# Use vectorizer to transform test data
X_test = tfidf_vectorizer.transform(df_test['tweet_cleaned'])
y_test = df_test["label"]

# Evaluate model on test set
y_test_pred = best_model.predict(X_test)
test_report = classification_report(y_test, y_test_pred, output_dict=True)
cm = confusion_matrix(y_test, y_test_pred)

#--------------------------------------
# Calculate additional metrics and add them to classification report df
accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred, average='weighted')
recall = recall_score(y_test, y_test_pred, average='weighted')
f1 = f1_score(y_test, y_test_pred, average='weighted')

classification_report_df = pd.DataFrame(test_report).transpose()

classification_report_df['accuracy'] = accuracy
classification_report_df['precision'] = precision
classification_report_df['recall'] = recall
classification_report_df['f1_score'] = f1
#----------------------

print("Test Classification Report:")
print(test_report)

# Save best model and hyperparameter search results
with open("svm_tfidf_poly_best_model_opt.pkl", "wb") as file:
    pickle.dump(best_model, file)

results_df = pd.DataFrame(random_search.cv_results_)
results_df.to_csv("svm_tfidf_poly_results_opt.csv", index=False)

# Save configuration of best model
best_model_config = best_model.get_params()
best_model_config_df = pd.DataFrame([best_model_config])
best_model_config_df.to_csv("svm_tfidf_poly_best_model_config_opt.csv", index=False)

# Save confusion matrix
unique_labels = sorted(y_test.unique())
cm_df = pd.DataFrame(cm, 
                     index=[f"Actual_{label}" for label in unique_labels],
                     columns=[f"Predicted_{label}" for label in unique_labels])
cm_df.to_csv("svm_tfidf_poly_confusion_matrix_opt.csv", index=True)

# Save classification report
classification_report_df = pd.DataFrame(test_report).transpose()
classification_report_df.to_csv("svm_tfidf_poly_classification_report_opt.csv", index=True)


Fitting 2 folds for each of 10 candidates, totalling 20 fits


In [7]:
# # Load training data
# df_train = pd.read_csv("../data/mixed_dataset/train_cleaned.csv")
# df_train = df_train.dropna(subset=['tweet_cleaned'])

# # TF-IDF Vectorization on training data + saving it 
# X_train, X_test, y_train, y_test, tfidf_vectorizer = vectorize_tfidf(df=df_train, text_column="tweet_cleaned", label_column="label")

# with open("svm_tfidf_rbf_vectorizer_opt.pkl", "wb") as file:
#     pickle.dump(tfidf_vectorizer, file)

# Hyperparameter search for SVC-model
param_dist = {
    'kernel': ['rbf'],
    'C': uniform(0.01, 1),
    'gamma': uniform(0.01, 0.1),
    'class_weight': [{0: 1, 1: 1}, {0: 1, 1: 3}, {0: 1, 1: 5}]
}

random_search = RandomizedSearchCV(
    SVC(),
    param_distributions=param_dist,
    n_iter=10, 
    scoring='f1',
    cv=2, 
    verbose=1,  
    n_jobs=-1,  
    random_state=42,
    refit=True 
)

random_search.fit(X_train, y_train)

# Retrieing best model found
best_model = random_search.best_estimator_

# Load test data
df_test = pd.read_csv("../data/mixed_dataset/test_cleaned.csv")
df_test = df_test.dropna(subset=['tweet_cleaned'])

# Use vectorizer to transform test data
X_test = tfidf_vectorizer.transform(df_test['tweet_cleaned'])
y_test = df_test["label"]

# Evaluate model on test set
y_test_pred = best_model.predict(X_test)
test_report = classification_report(y_test, y_test_pred, output_dict=True)
cm = confusion_matrix(y_test, y_test_pred)

#--------------------------------------
# Calculate additional metrics and add them to classification report df
accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred, average='weighted')
recall = recall_score(y_test, y_test_pred, average='weighted')
f1 = f1_score(y_test, y_test_pred, average='weighted')

classification_report_df = pd.DataFrame(test_report).transpose()

classification_report_df['accuracy'] = accuracy
classification_report_df['precision'] = precision
classification_report_df['recall'] = recall
classification_report_df['f1_score'] = f1
#----------------------

print("Test Classification Report:")
print(test_report)

# Save best model and hyperparameter search results
with open("svm_tfidf_rbf_best_model_opt.pkl", "wb") as file:
    pickle.dump(best_model, file)

results_df = pd.DataFrame(random_search.cv_results_)
results_df.to_csv("svm_tfidf_rbf_results_opt.csv", index=False)

# Save configuration of best model
best_model_config = best_model.get_params()
best_model_config_df = pd.DataFrame([best_model_config])
best_model_config_df.to_csv("svm_tfidf_rbf_best_model_config_opt.csv", index=False)

# Save confusion matrix
unique_labels = sorted(y_test.unique())
cm_df = pd.DataFrame(cm, 
                     index=[f"Actual_{label}" for label in unique_labels],
                     columns=[f"Predicted_{label}" for label in unique_labels])
cm_df.to_csv("svm_tfidf_rbf_confusion_matrix_opt.csv", index=True)

# Save classification report
classification_report_df = pd.DataFrame(test_report).transpose()
classification_report_df.to_csv("svm_tfidf_rbf_classification_report_opt.csv", index=True)


Fitting 2 folds for each of 10 candidates, totalling 20 fits
Test Classification Report:
{'0': {'precision': 0.9063684327859107, 'recall': 0.6977289628739775, 'f1-score': 0.7884801861788091, 'support': 34962.0}, '1': {'precision': 0.5186737110584806, 'recall': 0.818809318377912, 'f1-score': 0.6350658041490074, 'support': 13908.0}, 'accuracy': 0.7321874360548394, 'macro avg': {'precision': 0.7125210719221957, 'recall': 0.7582691406259447, 'f1-score': 0.7117729951639082, 'support': 48870.0}, 'weighted avg': {'precision': 0.7960337041224137, 'recall': 0.7321874360548394, 'f1-score': 0.7448197150253308, 'support': 48870.0}}
